In [1]:
# Required Packages to install
# ! pip install cmake --user
# ! pip install dlib --user
# ! pip install moviepy --user
# ! pip install ffmpeg --user

In [2]:
# First, set up the required folders

# os.mkdir('raw_images')
# os.mkdir('aligned_images')
# os.mkdir('generated_images')

In [1]:
import ffmpeg
import imageio
import os
import random
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline

tflib.init_tf()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [2]:
#This code used for pretrained Stylegan interpolations
URL_FFHQ = 'https://s3-us-west-2.amazonaws.com/nanonets/blogs/karras2019stylegan-ffhq-1024x1024.pkl'

with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs = pickle.load(f)

generator = Generator(Gs, batch_size=1, randomize_noise=False)

In [3]:
# #This code used for finetuned Stylegan interpolations
# a = os.getcwd()
# model = a + '/results/00003-sgan-dream1024-4gpu/network-snapshot-007860.pkl'

# with open(model, 'rb') as f:
#     generator_network, discriminator_network, Gs = pickle.load(f)
    
# generator = Generator(Gs, batch_size=1, randomize_noise=False)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ValueError: Cannot feed value of shape (1, 16, 512) for Tensor 'learnable_dlatents:0', which has shape '(1, 18, 512)'

In [ ]:
def generate_image_for_video(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    
    return img_array

In [ ]:
raw = 'FINEraw/' # set this to the directory path of your raw images
aligned = 'FINEaligned/' # set this to the path for aligned images
gen = 'FINEgen/' # set this to the path for GAN generations of aligned images
latent = 'FINElatent/' # set this to the path for latent (.npy) representations 

vid_range = 100 # this is used to determine number of frames between characters for interpolation videos, in practice this makes the transformations occur more slowly
vid_factor = 1/vid_range # used for determining step size between frames

In [ ]:
# Extracts and aligns faces from images
# !python align_images.py {raw} {aligned}

In [ ]:
#   Find latent representations of aligned images
#   This cell may take a long time (~1 hr) and it is recommended to run in terminal instead.
# import time
# start = time.time()

# !python encode_images.py {aligned} {gen} {latent}

# end = (time.time() - start)
# print(end)

In [ ]:
# This cell will load all latent representations from the latent directory into a list called chars
# This can then be used for video generation

path = os.getcwd() + "/" + latent
dirs = os.listdir( path )
chars = [] # this will be the list of all 
i = 0

for each in dirs:
    chars.append(np.load('%s/%s' %(latent, each)))

In [ ]:
# If you're interpolating between two characters, set these characters here
# char1 = chars[0]
# char2 = chars[1]

# This creates an nd array that stores all the image frames for cross-character interpolation
# video speed may be adjusted by changing the range,
# src_images = np.stack(generate_image_for_video((vid_factor*alpha*char2)+((1-(vid_factor*alpha))*char1)) for alpha in range (vid_range))

# imageio.mimwrite('tests2.mp4',src_images, fps = 20)

In [ ]:
# Interpolation between many characters, all the latent representations in the latent folder
random_walk = [i for i in range(len(chars))]
random.shuffle(random_walk)
frames = []
i = 0

while i < len(random_walk)-1:
    print(i)
    frames.extend(np.stack(generate_image_for_video((vid_factor*alpha*chars[random_walk[i+1]])+((1-(vid_factor*alpha))*chars[random_walk[i]])) for alpha in range (vid_range)))
    i+=1

imageio.mimwrite('testfinetuning.mp4',np.array(frames), fps = 20)

In [ ]:
# # Testing a looping interpolation
# chars = [jon, drogo, cersei]
# random_walk = [i for i in range(len(chars))]
# random.shuffle(random_walk)
# frames = []
# i = 0

# while i < len(random_walk)-1:
#     frames.extend(np.stack(generate_image_for_video((0.01*alpha*chars[random_walk[i+1]])+((1-(0.01*alpha))*chars[random_walk[i]])) for alpha in range (100)))
#     i+=1
    
# frames.extend(np.stack(generate_image_for_video((0.01*alpha*chars[random_walk[i]])+((1-(0.01*alpha))*chars[random_walk[0]])) for alpha in range (100)))

# #print(frames)
# imageio.mimwrite('testlooping.mp4',np.array(frames), fps = 20)

In [ ]:
# # Loading Got Char representations
# jon = np.load('latent_representations/jon.npy')
# jon_2 = np.load('latent_representations/jon_2.npy')
# tyrion = np.load('latent_representations/tyrion.npy')
# daenerys = np.load('latent_representations/daenerys.npy')
# drogo = np.load('latent_representations/drogo.npy')
# bran = np.load('latent_representations/bran.npy')
# cersei = np.load('latent_representations/cersei.npy')
# jaime = np.load('latent_representations/jaime.npy')
# theon = np.load('latent_representations/theon.npy')
# night_king_1 = np.load('latent_representations/night_king_1.npy')
# night_king_2 = np.load('latent_representations/night_king_2.npy')

# # Loading already learned representations
# a = np.load('latent_representations/a.npy') #Brandi
# b = np.load('latent_representations/b.npy')
# c = np.load('latent_representations/c.npy')
# d = np.load('latent_representations/d.npy')
# e = np.load('latent_representations/e.npy')
# f = np.load('latent_representations/f.npy')
# g = np.load('latent_representations/g.npy')
# h = np.load('latent_representations/h.npy')
# i = np.load('latent_representations/i.npy')
# j = np.load('latent_representations/j.npy')
# k = np.load('latent_representations/k.npy')
# l = np.load('latent_representations/l.npy')
# m = np.load('latent_representations/m.npy')
# n = np.load('latent_representations/n.npy')
# o = np.load('latent_representations/o.npy')
# p = np.load('latent_representations/p.npy')
# q = np.load('latent_representations/q.npy') #Brandi BM Hat

# ian = np.load('latent_representations/ian_01.npy')
# wolf = np.load('latent_representations/werewolf.npy')
# cedric = np.load('latent_representations/cedric_01.npy') 

# p1 = np.load('latent_representations/p1.npy')
# p2 = np.load('latent_representations/p2.npy')
# t1 = np.load('latent_representations/t1.npy')